In [1]:
%%time
from IPython.display import clear_output
!pip install ultralytics ultralytics-hub
!pip install roboflow
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install opencv-python
clear_output()

CPU times: user 248 ms, sys: 72.9 ms, total: 321 ms
Wall time: 17.4 s


In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="cBU8lFOC0t6ANNTPCksS")
project = rf.workspace("skincancerproject-85mbz").project("skin-cancer-rf9os")
version = project.version(2)
dataset = version.download("folder")
clear_output()


In [3]:
from ultralytics import YOLO
model = YOLO('yolov8m-cls.pt')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 32.7M/32.7M [00:01<00:00, 30.0MB/s]


In [4]:
import yaml

data_yaml = {
    "train": "/kaggle/working/Skin-Cancer-2/train",
    "val": "/kaggle/working/Skin-Cancer-2/valid",
    "nc": 2,  # Number of classes, update as per your dataset
    "names": ["benign", "malignant"]  # Replace with actual class names
}

with open("/kaggle/working/data.yaml", "w") as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print("data.yaml file created successfully.")


data.yaml file created successfully.


In [5]:
import os

train_path = "/kaggle/working/Skin-Cancer-2/train"
val_path = "/kaggle/working/Skin-Cancer-2/valid"

print("Train Path Exists:", os.path.exists(train_path))
print("Validation Path Exists:", os.path.exists(val_path))

# Check contents of the directories
print("Train Classes:", os.listdir(train_path) if os.path.exists(train_path) else "Path not found")
print("Validation Classes:", os.listdir(val_path) if os.path.exists(val_path) else "Path not found")


Train Path Exists: True
Validation Path Exists: True
Train Classes: ['benign', 'malignant']
Validation Classes: ['benign', 'malignant']


In [6]:
results = model.train(
    data='/kaggle/working/Skin-Cancer-2',
    epochs=30,
    imgsz=640,  # Consider reducing to 512 if acceptable for your task
    batch=32,  # Increased batch size for dual T4 GPUs
    optimizer='AdamW',
    lr0=0.001,
    dropout=0.2,
    project='skin_cancer_classification',
    name='yolov8n_cls',
    device=[0,1],  # Explicitly use both GPUs
    weight_decay=0.0005,
    amp=True,  # Crucial for T4 GPUs (they have Tensor Cores)
    workers=8,  # Kaggle works best with 4-8 workers
    cache='ram',  # Cache dataset in RAM if you have enough
    patience=10,  # Early stopping to avoid unnecessary epochs
    pretrained=True,  # Ensure you're using pretrained weights
    verbose=True
)

Ultralytics 8.3.97 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=/kaggle/working/Skin-Cancer-2, epochs=30, time=None, patience=10, batch=32, imgsz=640, save=True, save_period=-1, cache=ram, device=[0, 1], workers=8, project=skin_cancer_classification, name=yolov8n_cls, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=

In [7]:
from google.colab import files
files.download("/kaggle/working/skin_cancer_classification/yolov8n_cls/weights/best.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
metrics = model.val()
print(metrics.top1)  # top-1 accuracy
print(metrics.top5)  # top-5 accuracy

Ultralytics 8.3.97 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
YOLOv8m-cls summary (fused): 42 layers, 15,765,218 parameters, 0 gradients, 41.6 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /kaggle/working/Skin-Cancer-2/train... found 79370 images in 2 classes ✅ 
val: /kaggle/working/Skin-Cancer-2/test... found 529 images in 2 classes ✅ 
test: /kaggle/working/Skin-Cancer-2/test... found 529 images in 2 classes ✅ 
WARNING ⚠️ Classification `cache_ram` training has known memory leak in https://github.com/ultralytics/ultralytics/issues/9824, setting `cache_ram=False`.


val: Scanning /kaggle/working/Skin-Cancer-2/test... 529 images, 0 corrupt: 100%|██████████| 529/529 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 17/17 [00:08<00:00,  1.94it/s]


                   all      0.941          1
Speed: 0.4ms preprocess, 12.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to skin_cancer_classification/yolov8n_cls2
0.9413988590240479
1.0


In [9]:
print(metrics.box.map)  # Print mAP (mean Average Precision)

AttributeError: 'ClassifyMetrics' object has no attribute 'box'. See valid attributes below.

    Class for computing classification metrics including top-1 and top-5 accuracy.

    Attributes:
        top1 (float): The top-1 accuracy.
        top5 (float): The top-5 accuracy.
        speed (dict): A dictionary containing the time taken for each step in the pipeline.
        task (str): The task type, set to 'classify'.
    

In [ ]:
import cv2
import glob
import os

# Load the trained model
model = YOLO("runs/detect/yolo11n_finetuned2/weights/best.pt")  # Update with your trained model path

# Path to the folder containing test images
image_folder = "/content/Implant-Doctor-1/test/images"  # Change this to your test image folder
output_folder = "./output"  # Change this to where you want to save results

os.makedirs(output_folder, exist_ok=True)

# Get a list of images (limit to 6 images)
image_paths = glob.glob(os.path.join(image_folder, "*.jpg"))[:10]

# Run inference on images
for img_path in image_paths:
    results = model(img_path)  # Run YOLO model on the image

    # Save the result image with bounding boxes
    for result in results:
        img = result.plot()  # Get the image with detections
        output_path = os.path.join(output_folder, os.path.basename(img_path))
        cv2.imwrite(output_path, img)

print("Inference completed. Results saved in:", output_folder)